In [24]:
import pandas as pd
import ast
import itertools
import numpy as np
from surprise import SVD, KNNBasic, NMF
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as sp




In [73]:
animes = pd.read_csv('data/animes.csv')
profiles = pd.read_csv('data/profiles.csv')


### Part 1: Naive exploration

In [26]:
animes.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [27]:
profiles.head()

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99


In [28]:
profiles.describe()

,profile,gender,birthday,favorites_anime,link
count,81727,53856,46807,81727,81727
unique,47885,3,7708,35395,47885
top,Voivodian,Male,1995,[],https://myanimelist.net/profile/Voivodian
freq,6,37096,211,16602,6


In [29]:
anime_list = ['Fullmetal Alchemist: Brotherhood','Haikyuu!! Second Season']
anime_ids = list(set(animes.loc[animes['title'].isin(anime_list), 'uid']))
anime_ids = list(map(str,anime_ids))
anime_ids

['5114', '28891']

In [30]:
def safe_literal_eval(node):
     try:
         return ast.literal_eval(node)
     except ValueError:
         return ['nope']  # happens when literal eval cannot process node

In [31]:
scores = []
similars = []
profiles = profiles[profiles['favorites_anime']!='[]']
profiles.head()

,profile,gender,birthday,favorites_anime,link
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn
3,edgewalker00,Male,Sep 5,"['5680', '849', '2904', '3588', '37349']",https://myanimelist.net/profile/edgewalker00
4,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99


In [32]:
profiles_list = profiles['favorites_anime'].apply(safe_literal_eval)


for item in profiles_list:
    
    intersection = list(set(item) & set(anime_ids))
    score = len(intersection)/max(len(item),len(anime_ids))
    scores.append(score)

    similars.append(item)
df = pd.DataFrame(list(zip(scores,similars)),columns=['Score','Anime'])

In [33]:
df_reco = df[~df['Anime'].isin(anime_ids)].sort_values(by='Score', ascending=False)
df_reco['len'] = df_reco['Anime'].apply(len)
df_reco = df_reco[df_reco['len']!=1].head(20)
df_reco

,Score,Anime,len
52728,0.666667,"[28735, 5114, 28891]",3
8452,0.500000,"[5114, 18679]",2
30935,0.500000,"[1575, 5114]",2
51768,0.500000,"[9756, 5114]",2
29161,0.500000,"[28891, 35075]",2
33626,0.500000,"[270, 5114]",2
5082,0.500000,"[5114, 37779]",2
57961,0.500000,"[5114, 21]",2
61570,0.500000,"[5114, 853]",2
36535,0.500000,"[5114, 9330]",2


In [34]:
reco_ids = list(set(list(itertools.chain(*df_reco['Anime'].values.tolist()))))
reco_ids = list(map(int,reco_ids))

In [35]:
reco_names = animes.loc[animes['uid'].isin(reco_ids), 'title'].unique()
list(set(reco_names).difference(anime_list))

['Mahou Shoujo Madoka★Magica',
 'Gintama',
 'Fairy Tail: Final Series',
 'Monster',
 'Sen to Chihiro no Kamikakushi',
 'Berserk',
 'Magic Kaito 1412',
 'Steins;Gate',
 'Shouwa Genroku Rakugo Shinjuu',
 'Code Geass: Hangyaku no Lelouch',
 'Dragon Crisis!',
 'Fullmetal Alchemist',
 'Yakusoku no Neverland',
 'Hellsing',
 'Ouran Koukou Host Club',
 'Hoozuki no Reitetsu 2nd Season',
 'Kill la Kill',
 'Death Note',
 'One Piece']

In [36]:
def get_recos(anime_list, n=20):
    anime_ids = list(set(animes.loc[animes['title'].isin(anime_list), 'uid']))
    anime_ids = list(map(str,anime_ids))
    scores = []
    similars = []
    profiles_df = profiles[profiles['favorites_anime']!='[]']
    profiles_list = profiles['favorites_anime'].apply(safe_literal_eval)
    for item in profiles_list:

        intersection = list(set(item) & set(anime_ids))
        score = len(intersection)/max(len(item),len(anime_ids))
        scores.append(score)

        similars.append(item)
        
    df = pd.DataFrame(list(zip(scores,similars)),columns=['Score','Anime'])
    df_reco = df[~df['Anime'].isin(anime_ids)].sort_values(by='Score', ascending=False)
    df_reco['len'] = df_reco['Anime'].apply(len)
    df_reco = df_reco[df_reco['len']!=1].head(n)
    reco_ids = list(set(list(itertools.chain(*df_reco['Anime'].values.tolist()))))
    reco_ids_list = list(map(int,reco_ids))
    reco_names = animes.loc[animes['uid'].isin(reco_ids_list), 'title'].unique()
    reco_list = list(set(reco_names).difference(anime_list))
    return reco_list

    
    
    

In [37]:
get_recos(anime_list)

['Mahou Shoujo Madoka★Magica',
 'Gintama',
 'Fairy Tail: Final Series',
 'Monster',
 'Sen to Chihiro no Kamikakushi',
 'Berserk',
 'Magic Kaito 1412',
 'Steins;Gate',
 'Shouwa Genroku Rakugo Shinjuu',
 'Code Geass: Hangyaku no Lelouch',
 'Dragon Crisis!',
 'Fullmetal Alchemist',
 'Yakusoku no Neverland',
 'Hellsing',
 'Ouran Koukou Host Club',
 'Hoozuki no Reitetsu 2nd Season',
 'Kill la Kill',
 'Death Note',
 'One Piece']

### Part 2: Let's get real: Rec. Sys. with SVD_

In [38]:
reviews = pd.read_csv('data/reviews.csv',usecols=['uid','anime_uid','score'])
reviews.head(10)
reviews.to_csv('data/reviews_light.csv')


In [39]:
#table = pd.pivot_table(reviews, values='score', index='profile', columns=['anime_uid'], aggfunc=np.mean)

In [40]:

min_anime_ratings = 500
filter_anime = reviews['anime_uid'].value_counts() > min_anime_ratings
filter_anime = filter_anime[filter_anime].index.tolist()

min_user_ratings = 1
filter_users = reviews['uid'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df = reviews[(reviews['anime_uid'].isin(filter_anime)) & (reviews['uid'].isin(filter_users))]
df.head()



,uid,anime_uid,score
1,259117,34599,10
3,8254,2904,9
4,291149,4181,10
5,10046,2904,10
7,140903,2904,8


#### Now, adding a new set of reviews

In [41]:
new_uid = max(df['uid'])+1
anime_list = ['Fullmetal Alchemist', 'Gintama', 'Death Note']
new_anime_ids = animes.loc[animes['title'].isin(anime_list), 'uid'].unique()
new_list = list(zip([new_uid]*3,list(new_anime_ids),list(10*np.ones(len(anime_list)))))
new_df = pd.DataFrame(new_list, columns=['uid','anime_uid','score'])
df_reviews = df.append(new_df).reset_index(drop=True)
df_reviews['score'] = df_reviews['score'].apply(int)

df_reviews

C:\Users\EmileESMAILI\AppData\Local\Temp\ipykernel_20820\3015116004.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_reviews = df.append(new_df).reset_index(drop=True)


,uid,anime_uid,score
0,259117,34599,10
1,8254,2904,9
2,291149,4181,10
3,10046,2904,10
4,140903,2904,8
...,...,...,...
38424,93509,10620,9
38425,50225,10620,10
38426,325671,918,10
38427,325671,121,10


#### surprise SVD

In [42]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_reviews[['uid', 'anime_uid', 'score']], reader)

In [43]:
# Use the famous SVD algorithm.
algo = SVD(random_state=42)

# Run k-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.1565  1.1905  1.1500  1.1656  0.0178  
MAE (testset)     0.6918  0.7105  0.6976  0.7000  0.0078  
Fit time          2.62    2.32    2.38    2.44    0.13    
Test time         0.27    0.13    0.13    0.18    0.07    


{'test_rmse': array([1.15648355, 1.19045142, 1.14996375]),
 'test_mae': array([0.69176793, 0.71050731, 0.69758211]),
 'fit_time': (2.6183040142059326, 2.317638874053955, 2.3817272186279297),
 'test_time': (0.27468371391296387, 0.13465332984924316, 0.12736749649047852)}

In [44]:
# Retrieve the trainset.
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
algo = SVD()
algo.fit(trainset)

In [45]:
uid = 325671  # raw user id (as in the ratings file).
iid = 5514  # raw item id (as in the ratings file).

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)

user: 325671     item: 5514       r_ui = None   est = 8.55   {'was_impossible': False}


In [46]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [47]:
# Then predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

top_n[uid]

KeyboardInterrupt: 

In [ ]:
# Print the recommended items for each user
anime_ids = []
scores = []
for item in top_n[uid]:
    anime_ids.append(item[0])
    scores.append(item[1])
    df = pd.DataFrame(list(zip(list(anime_ids),scores)),columns=['anime_id','score'])

In [ ]:
df

,anime_id,score
0,5114,9.794066
1,4181,9.610972
2,2904,9.575576
3,4224,9.548405
4,853,9.475180
5,2001,9.332091
6,1210,9.312071
7,21,9.254412
8,30276,9.162919
9,5081,9.086108


### Part 3: Genre based score

In [215]:

animes = animes.drop_duplicates(subset='uid').reset_index()
animes.describe()

,index,uid,episodes,members,popularity,ranked,score
count,16216.000000,16216.000000,15724.000000,1.621600e+04,16216.000000,14553.000000,15875.000000
mean,8716.190614,19858.979588,11.657975,2.759189e+04,8161.171621,7332.375318,6.346466
std,5241.776670,14197.712860,48.973125,9.388251e+04,4714.317489,4235.759134,0.972700
min,0.000000,1.000000,1.000000,2.500000e+01,1.000000,1.000000,1.250000
25%,4160.750000,5303.750000,1.000000,2.920000e+02,4082.000000,3659.000000,5.700000
50%,8649.000000,19856.000000,2.000000,1.839500e+03,8158.500000,7337.000000,6.370000
75%,12818.250000,34086.500000,12.000000,1.160000e+04,12243.250000,10993.000000,7.040000
max,19006.000000,40960.000000,3057.000000,1.871043e+06,16338.000000,14675.000000,9.230000


In [216]:
animes["genre"] = animes["genre"].apply(ast.literal_eval)

ValueError: malformed node or string: ['Comedy', 'Sports', 'Drama', 'School', 'Shounen']

In [217]:
animes['genre'][0]

['Comedy', 'Sports', 'Drama', 'School', 'Shounen']

In [218]:
animes['genre_list'] = animes['genre'].apply(lambda x: ', '.join(x))

In [79]:
animes['genre_list'][2]

'Sci-Fi, Adventure, Mystery, Drama, Fantasy'

In [219]:
anime = ['Monster','Fullmetal Alchemist: Brotherhood','Cowboy Bebop','Hunter x Hunter','Yu☆Gi☆Oh!']
idx = animes[animes['title'].isin(anime)].index
print(idx)
anime = list(animes.iloc[idx]['title'].values)
anime


Int64Index([3, 673, 744, 764, 15270], dtype='int64')


['Fullmetal Alchemist: Brotherhood',
 'Hunter x Hunter',
 'Monster',
 'Cowboy Bebop',
 'Yu☆Gi☆Oh!']

In [208]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(animes['genre_list'])
Xc = (X[idx,:]* X.T).toarray()
#Xc_norm = Xc * (1./int(Xc[:,idx]))
#g = sp.diags(1./Xc.T.diagonal())
#Xc_norm = g * Xc # normalized co-occurence matrix
Xc

array([[3, 2, 3, ..., 2, 1, 2],
       [1, 1, 2, ..., 1, 1, 0],
       [1, 1, 2, ..., 0, 0, 1],
       [2, 1, 4, ..., 1, 1, 2]], dtype=int64)

In [209]:
df = pd.DataFrame(Xc.T, index=animes['title'],columns=anime)
mask = df.index.isin(anime)
df_masked = df[~mask]
df_norm = df_masked / df_masked.max()
df_norm.head()

,Fullmetal Alchemist: Brotherhood,Hunter x Hunter,Monster,Cowboy Bebop
title,,,,
Haikyuu!! Second Season,0.375,0.166667,0.2,0.285714
Shigatsu wa Kimi no Uso,0.250,0.166667,0.2,0.142857
Made in Abyss,0.375,0.333333,0.4,0.571429
Kizumonogatari III: Reiketsu-hen,0.125,0.166667,0.2,0.142857
Mob Psycho 100 II,0.250,0.166667,0.0,0.285714


In [200]:
animes[animes['title'].isin(anime+['Mob Psycho 100 II'])][['title','genre_list']]

,title,genre_list
3,Fullmetal Alchemist: Brotherhood,"Action, Military, Adventure, Comedy, Drama, Ma..."
5,Mob Psycho 100 II,"Action, Slice of Life, Comedy, Supernatural"
744,Monster,"Drama, Horror, Mystery, Police, Psychological,..."
764,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space"


In [211]:
df_norm['average'] = df_norm.mean(axis=1)
reco_genre = df_norm.sort_values(by='average',ascending=False)
reco_genre

,Fullmetal Alchemist: Brotherhood,Hunter x Hunter,Monster,Cowboy Bebop,average
title,,,,,
One Piece: Long Ring Long Land-hen,0.75,1.0,0.2,0.571429,0.630357
One Piece Movie 14: Stampede,0.75,1.0,0.2,0.571429,0.630357
One Piece: Episode of Sorajima,0.75,1.0,0.2,0.571429,0.630357
One Piece: Episode of Sabo - 3 Kyoudai no Kizuna Kiseki no Saikai to Uketsugareru Ishi,0.75,1.0,0.2,0.571429,0.630357
One Piece: Episode of Merry - Mou Hitori no Nakama no Monogatari,0.75,1.0,0.2,0.571429,0.630357
...,...,...,...,...,...
Yume,0.00,0.0,0.0,0.000000,0.000000
Spocon!,0.00,0.0,0.0,0.000000,0.000000
Miracle! Mimika,0.00,0.0,0.0,0.000000,0.000000
